# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import cassandra
import os
import glob
import csv
import pandas as pd

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('all_event_data.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('all_event_data.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II.

## Now you are ready to work with the CSV file titled <font color=red>all_event_data.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**all_event_data.csv**</font> after the code above is run:<br>

<img src="images/all_event_data.jpg">

#### Creating a Cluster

In [5]:
# Connecting to the Cassandra instance

from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating a keyspace if not exist to work with 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Connecting to the keyspace we created
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries.

## Create queries to ask the following three questions of the data

### 1. Give me the `artist`, `song title` and `song's length` in the music app history that was heard during  `sessionId = 338`, and `itemInSession  = 4`


### 2. Give me only the following: `name of artist`, `song` (`sorted by itemInSession`) and `user` (`first` and `last` name) for `userid` = 10, `sessionid` = 182
    

### 3. Give me every `user name` (first and last) in my music app history who listened to the `song` 'All Hands Against His Own'




---
# 1. Creating Tables

## Question1:
- **Give me the `artist`, `song title` and `song's length` in the music app history that was heard during  `sessionId = 338`, and `itemInSession  = 4`**

**Since we are working with NoSQL database we need to think about the queries we want to perfom to the data so, In this question we are asked to retrieve the (artist, song title, song's length) from data based on specific sessionId and itemInSession. we can specify (sessionId and itemInSession) as out primary key as it can uniquely identifiy each row and
we need to filter the data based on them and we can add the rest of the rest of the information of the query as table columns and our table will look like this:**

## Table1

`Table Name: sessions
column 1: sessionId
column 2: itemInSession
column 3: artist
Column 4: song_title
Column 5: song_length
PRIMARY KEY(SessionId,itemInSession)`

In [8]:
# query1 to create session_history Table
query = "CREATE TABLE IF NOT EXISTS sessions "
query = query + "(sessionId INT, itemInSession INT, artist TEXT, song_title TEXT, song_length FLOAT, PRIMARY KEY(SessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

## Query2:
- **Give me only the following: `name of artist`, `song` (`sorted by itemInSession`) and `user` (`first` and `last` name) for `userid` = 10, `sessionid` = 182**

**In this question we are asked to retrive (artist name, song title (sorted by itemInSession) and user name) for a specific userId and sessionId, best way to partion this information is by using (userId & sessionId) as a composite primary key to make sure they partioned together and adding itemInSession as a clustering column to sort the data. and the table has to include artist name, song title and user name as well, so the second table may look like this**

## Table2

`Table Name: users
column 1: userId
column 2: sessionId
column 3: itemInSession
Column 4: artist
Column 5: song
Column 6: userName
PRIMARY KEY(userId,sessionId,itemInSession)`

In [9]:
# query2 to create users table
query = "CREATE TABLE IF NOT EXISTS users "
query = query + "(userId INT, sessionId INT, itemInSession INT, artist TEXT, song TEXT, userName TEXT, PRIMARY KEY((userId, sessionId), ItemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

## Query3:
- **Give me every `user name` (first and last) in my music app history who listened to the `song` 'All Hands Against His Own'**

**Here we are asked to retrieve the name of the user that listened to a specific song title. since song title is our filtring columns we use it as a primary key, but if used soely it may have duplicated records to solve that we can add userId to ensure the uniquenese of each row.**

## Table3

`Table Name: songs
column 1: song
column 2: userId
Column 3: user_name
PRIMARY KEY(song, userId, sessionId)`

In [10]:
# query3 to create songs table
query = "CREATE TABLE IF NOT EXISTS songs "
query = query + "(song TEXT, userId INT, user_name TEXT, PRIMARY KEY(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)  
    

---
# 2. Inserting Data

In [11]:
file = 'all_event_data.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Inserting into sessions
        query = "INSERT INTO sessions (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])) )
        
        # Inserting into users
        query = "INSERT INTO users (userId, sessionId, itemInSession, artist, song, userName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1]+' '+line[4]))
        
        # Inserting into songs
        query = "INSERT INTO songs (song, userId, user_name)"
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1]+' '+line[4]))

---
# 3. Testing Queries

### Query1:
We would like to retrieve (artist,song title,song length) from table session where sessionId is 338 and itemInSession is 4. 
- The query will look like this:

```
SELECT
    artist,
    song_title,
    song_length
FROM sessions
WHERE sessionId = 338  AND itemInSession = 4
```

In [12]:
# Query1 Structure SELECT artist,song_title,song_length FROM sessions WHERE sessionId = value AND itemInSession = value
query = """
SELECT
    artist,
    song_title,
    song_length
FROM sessions
WHERE sessionID = 338  AND itemInSession = 4"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#for row in rows:
#    print(row.artist,'|', row.song_title,'|', row.song_length)
result = pd.DataFrame(columns=rows.column_names)
for row in rows:
    result = result.append(other= pd.Series([row.artist,row.song_title,row.song_length],index=rows.column_names), ignore_index=True)
result.head()

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query2:
We would like to retrieve (artist,song title,user name) from table users where userId is 10 and sessionId is 182. 
- The query will look like this:

```
SELECT
    artist,
    song,
    userName
FROM users
WHERE userId = 10  AND sessionId = 182
```

In [13]:
query = """
SELECT
    artist,
    song,
    userName
FROM users
WHERE userId = 10  AND sessionId = 182"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#for row in rows:
#    print(row.artist,'|', row.song,'|', row.username)
result = pd.DataFrame(columns=rows.column_names)
for row in rows:
    result = result.append(other= pd.Series([row.artist,row.song,row.username],index=rows.column_names), ignore_index=True)
result.head()                

,artist,song,username
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


### Query3:
We would like to retrieve the name of the users that lisitened to the song 'All Hands Against His Own'
- The select statment will look like this:

```
SELECT user_name
FROM songs
WHERE song = 'All Hands Against His Own' 
```

In [14]:
query = """
SELECT user_name
FROM songs
WHERE song = 'All Hands Against His Own' """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#for row in rows:
#    print(row.user_name)           
result = pd.DataFrame(columns=rows.column_names)
for row in rows:
    result = result.append(other= pd.Series([row.user_name],index=rows.column_names), ignore_index=True)
result.head()  

,user_name
0,Jacqueline Lynch
1,Tegan Levine
2,Sara Johnson


### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()